SPODNJA CELICA JE SAMO ZA PRIKAZ NELINEARNE TRIGONOMETRIČNE ENAČBE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def det(x):
    return np.cosh(x)*np.cos(x)+1

x = np.linspace(0, 30, 100)
plt.plot(x, det(x), label='cosh(x)*cos(x) + 1')
plt.xlabel('x')
plt.title('Nelinearna trigonometrična funkcija')
plt.legend()
plt.grid(True)
plt.xlim(left=0)

plt.savefig('graf.png', dpi=300)
plt.show()

Podatke sem ocenu preko te spletne strani: https://www.simplify3d.com/resources/materials-guide/properties-table/
SPODNJA CELICA JE ZA ISKANJE REŠITEV PA PRIKAZ NA ENEM PRIMERU

In [ ]:
# Uvozimo vse potrebne pakete in funkcije
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from ipywidgets import interact, IntSlider, FloatSlider
from matplotlib.animation import FuncAnimation


# Valovno število k_n pomnoženo z dolžino
def det(x):
    return np.cosh(x)*np.cos(x)+1

# Definiramo območje, v katerem iščemo rešitve
x0 = np.linspace(0, 150, 100)

# seznam rešitev
r = []

# for zanka za iskanje rešitve
for i in x0:
    res = root(det, i)
    if res.success:
        root_res = res.x[0]
        # Definiramo unikatne rešitve, kjer mora biti razlika med rešitvami > 1
        if not any(np.isclose(root_res, j, atol = 1) for j in r):
            r.append(root_res)
            # Ko imamo 5 rešitev, prekinemo zanko
            if len(r) >=5:
                break

print(f'Prvih 5 k_n*L je:')
for k_nL in r:
    print(f'{k_nL:0.5f} [1/m * m]')


# Lastne frekvence ω_n -> mora biti v osnovnih enotah, če ne pride do stack overflowa
togost = 50*10**6 # Youngov modul [Pa]
a = 70/1000 # stranica kvadrata prečnega prereza [m]
I = (a**4)/12 # vztrajnostni moment prečnega prereza nosilca [m^4]
μ = (1.2*1000)*a**2 # masna porazdelitev na enoto dolžine [kg/m]
L = 150/1000 # dolžina nosilca [m]

frekvence = []

for i in r:
    f = np.sqrt(((i/L)**4 * togost * I)/ μ)
    frekvence.append(f)

print(f'Prvih 5 lastnih frekvenc je:')
for ω_n in frekvence:
    print(f'{ω_n:0.5f} [Hz]')


# Neznani koeficienti A_i

A = []

for k_nL in r:
    A_i = -(np.sinh(k_nL) + np.sin(k_nL))/(np.cosh(k_nL)+np.cos(k_nL))
    A.append(A_i)

print(f'Koeficienti A_i so:')
for i in A:
    print(f'{i:0.5f}')

matrika = np.column_stack((A, frekvence))


# Upogib nosilca
def upogib(x, t, A_n, omega_n, μ=μ, I=I, togost=togost):
    enačbe = 0
    B = 1
    for A, omega in zip(A_n, omega_n):
        k = ((μ*omega**2)/togost*I)**0.25
        en = (A*np.cosh(k*x) + B*np.sinh(k*x) - A*np.cos(k*x) - B*np.sin(k*x))*(np.cos(omega*t) + np.sin(omega*t))
        enačbe += en
    return enačbe*1000

upogib(L, 10, A, frekvence)

# Prikaz
x = np.linspace(0, L, 100)
y_min = -2
y_max = 2

# Definiranje grafa, ki se spreminja po času
def graf(t):
    y = upogib(x, t, A, frekvence)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x*1000, y, label=f't = {t} s')
    plt.xlabel('x [mm]')
    plt.ylabel('w [mm]')
    plt.title('Upogib glede na čas')
    plt.xlim(0, (L*1000))
    plt.xticks(np.arange(0, L*1000+10, 10)) 
    plt.ylim(y_min, y_max)
    plt.legend()
    plt.grid(True)
    plt.show()

# Slider s korakom 0.01
interact(graf, t=FloatSlider(min=0, max=20, step=0.01, value=0))

SPODNJA CELICA JE ZA GENERIRANJE PODATKOV

In [7]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from ipywidgets import interact, IntSlider, FloatSlider
from matplotlib.animation import FuncAnimation

# Valovno število k_n pomnoženo z dolžino
def det(x):
    return np.cosh(x)*np.cos(x)+1

# Definiramo območje, v katerem iščemo rešitve
x0 = np.linspace(0, 150, 100)

# seznam rešitev
r = []

# for zanka za iskanje rešitve
for i in x0:
    res = root(det, i)
    if res.success:
        root_res = res.x[0]
        # Definiramo unikatne rešitve, kjer mora biti razlika med rešitvami > 10^-3
        if not any(np.isclose(root_res, j, atol = 1e-3) for j in r):
            r.append(root_res)
            # Ko imamo 5 rešitev, prekinemo zanko
            if len(r) >=5:
                break

# Generiranje podatkov
dolžine = (np.random.uniform(100, 501, 100))/1000 # [m]
togosti = (np.random.uniform(40, 81, 100))*10**6 # [Pa]
masne_porazdelitve = (np.random.uniform(1, 4.1, 100)*1000) # [kg/m^3]
stranice_kvadrata = np.random.uniform(10, 500, 100)/1000 # [m]
vztrajnostni_momenti = (stranice_kvadrata**4) / 12 # [m^4]

# Ustvarjanje matrike za frekvence
matrika_frekvenc = np.zeros((100, 5))

# Funkcija za izračun lastnih frekvenc
def izračun_frekvenc(L, E, mu, I):
    frekvence = []
    for i in r:
        f = np.sqrt(((i / L)**4 * E * I) / mu)
        frekvence.append(f)
    return frekvence

# Izračun frekvenc za vsak niz parametrov
for i in range(100):
    L = dolžine[i]
    E = togosti[i]
    a = stranice_kvadrata[i]
    mu = masne_porazdelitve[i] * (a ** 2)
    I = vztrajnostni_momenti[i]
    matrika_frekvenc[i, :] = izračun_frekvenc(L, E, mu, I)

np.shape(matrika_frekvenc)


# Funkcija za izračun upogiba nosilca
def upogib(x, t, A_n, omega_n, μ, I, togost):
    enačbe = 0
    B = 1
    for A, omega in zip(A_n, omega_n):
        k = ((μ * omega**2) / togost * I)**0.25
        en = (A * np.cosh(k * x) + B * np.sinh(k * x) - A * np.cos(k * x) - B * np.sin(k * x)) * (np.cos(omega * t) + np.sin(omega * t))
        enačbe += en
    return enačbe * 1000

# Funkcija za iskanje absolutnega maksimalnega upogiba v določenem časovnem obdobju
def iskanje_maks_upogib(x, t_min, t_max, A, omega, μ, I, togost):
    t_točke = np.linspace(t_min, t_max, 100)  # Iskanje znotraj prvih 100s
    maks_upogib = np.max([np.abs(upogib(x, t, A, omega, μ, I, togost)) for t in t_točke])
    return maks_upogib

# Generiranje matrike maksimalnih upogibov
matrika_maks_upogibov = np.zeros((100, 10))

# Izračun maksimalnega upogiba za vsak niz parametrov in za vsakih 10 različnih točk
for i in range(100):
    L = dolžine[i]
    E = togosti[i]
    a = stranice_kvadrata[i]
    mu = masne_porazdelitve[i] * (a ** 2)
    I = vztrajnostni_momenti[i]
    frekvence = matrika_frekvenc[i, :]
    
    A = [-(np.sinh(k_nL) + np.sin(k_nL)) / (np.cosh(k_nL) + np.cos(k_nL)) for k_nL in r]

    for j in range(1, 11):
        x_točka = L / j
        maks_upogib = iskanje_maks_upogib(x_točka, 0, 20, A, frekvence, mu, I, E)
        matrika_maks_upogibov[i, j-1] = maks_upogib

# Prikaz rezultatov
print(matrika_maks_upogibov)

Matrika maksimalnih upogibov (vsaka vrstica ustreza različnim parametrom, vsaka stolpec različni točki):
[[4.87676458e+01 1.24334092e+01 5.56173801e+00 3.13854033e+00
  2.01252989e+00 1.39937913e+00 1.02905407e+00 7.88408599e-01
  6.23271410e-01 5.05064513e-01]
 [7.68487025e-01 1.92597991e-01 8.56696602e-02 4.82090270e-02
  3.08613970e-02 2.14350534e-02 1.57500537e-02 1.20596979e-02
  9.52930347e-03 7.71915913e-03]
 [1.54880110e-01 3.87632344e-02 1.72345052e-02 9.69620948e-03
  6.20626538e-03 4.31022656e-03 3.16686502e-03 2.42472748e-03
  1.91589333e-03 1.55191200e-03]
 [7.74041659e+02 2.14796022e+02 9.86251038e+01 5.63656816e+01
  3.64154628e+01 2.54465886e+01 1.87784077e+01 1.44248530e+01
  1.14266882e+01 9.27458677e+00]
 [7.54700565e+02 2.07178565e+02 9.48257408e+01 5.41120892e+01
  3.49284399e+01 2.43932272e+01 1.79936428e+01 1.38177774e+01
  1.09431888e+01 8.88046726e+00]
 [3.65230790e+01 9.28942362e+00 4.15213740e+00 2.34218800e+00
  1.50153883e+00 1.04391054e+00 7.67571423e-01 5

SPODNJA CELICA JE ZA TRENIRANJE MODELA IN NAPOVEDOVANJE

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Definiramo funkcijo za tvorjenje modela
def CNN_model():
    # Definiramo obliko input parametrov
    pomiki_oblika = (10, 1)
    prvih_5_lastnih_frekvenc_oblika = (5, 1)
    vztrajnostni_moment_oblika = (1,)

    # Ustvarjanje input slojev
    input_pomiki = tf.keras.Input(shape=pomiki_oblika)
    x = layers.Conv1D(32, 3, activation='tanh')(input_pomiki)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(64, 3, activation='tanh')(x)
    x = layers.GlobalMaxPooling1D()(x)
    cnn_output_pomiki = layers.Dense(64, activation='tanh')(x)

    input_prvih_5_lastnih_frekvenc = tf.keras.Input(shape=prvih_5_lastnih_frekvenc_oblika)
    y = layers.Conv1D(32, 1, activation='relu')(input_prvih_5_lastnih_frekvenc)
    y = layers.MaxPooling1D(1)(y)
    y = layers.Conv1D(64, 1, activation='relu')(y)
    y = layers.GlobalMaxPooling1D()(y)
    cnn_output_frekvenc = layers.Dense(64, activation='relu')(y)

    input_vztrajnostni_moment = tf.keras.Input(shape=vztrajnostni_moment_oblika)
    dense_vztrajnostni_moment = layers.Dense(32, activation='relu')(input_vztrajnostni_moment)

    # Povezave med inputi layerji in tvorjenje outputa
    povezave = layers.Concatenate()([cnn_output_pomiki, cnn_output_frekvenc, dense_vztrajnostni_moment])
    output_dolzina = layers.Dense(1, name="dolzina")(povezave)
    output_togost = layers.Dense(1, name="togost")(povezave)

    # Definiranje modela
    model = models.Model(inputs=[input_pomiki, input_prvih_5_lastnih_frekvenc, input_vztrajnostni_moment], outputs=[output_dolzina, output_togost])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Ustvarjanje in pregled modela
model = CNN_model()
model.summary()


# Output oznake oz. naše ciljne vrednosti
Y_dolžina = dolžine
Y_togost = togosti

# Priprava ciljev (labels) za treniranje modela
Y_dolžina = Y_dolžina.reshape((100, 1))
Y_togost = Y_togost.reshape((100, 1))

# Treniranje modela
model.fit([matrika_maks_upogibov, matrika_frekvenc, vztrajnostni_momenti], [Y_dolžina, Y_togost], epochs=10, validation_split=0.2)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 10, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 5, 1)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 8, 32)             │             128 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 5, 32)             │              64 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d (MaxPooling1D)  │ (None, 4, 32)             │               0 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_1               │ (None, 5, 32)             │               0 │ conv1d_2[0][0]             │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 2, 64)             │           6,208 │ max_pooling1d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_3 (Conv1D)             │ (None, 5, 64)             │           2,112 │ max_pooling1d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d          │ (None, 64)                │               0 │ conv1d_1[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d_1        │ (None, 64)                │               0 │ conv1d_3[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_2 (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 64)                │           4,160 │ global_max_pooling1d[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           4,160 │ global_max_pooling1d_1[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │              64 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 160)               │               0 │ dense[0][0],               │
│                               │                           │                 │ dense_1[0][0],             │
│                               │                           │               

 Total params: 17,218 (67.26 KB)

 Trainable params: 17,218 (67.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 444ms/step - loss: 3711025153048576.0000 - val_loss: 3579964226011136.0000
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 3629768398340096.0000 - val_loss: 3579542245474304.0000
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 3687051719344128.0000 - val_loss: 3579108990648320.0000
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 3863578096435200.0000 - val_loss: 3578670903984128.0000
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 3689358653652992.0000 - val_loss: 3578230132965376.0000
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 3681950908809216.0000 - val_loss: 3577750438805504.0000
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 3746176776011776.0000 - val_loss: 3577233700552704.0000
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 3792004613931008.0000 - val_loss: 3576647437516800.0000
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 3769117270081536.0000 - val_loss: 357